<a href="https://colab.research.google.com/github/yschan8137/StockAnalytics/blob/master/TRivots_U_L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


#預載套件


In [2]:
#!pip install dash-bootstrap-components jupyter-dash plotly -q > log.txt
!pip install shioaji --upgrade > log.txt
#!pip install --upgrade shioaji > log.txt
#!pip install plotly --upgrade > log.txt
!pip install --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]' > log.txt
#!pip install talib-binary > log.txt
#!pip install yfinance > log.txt
!pip install --upgrade google-api-python-client > log.txt
!pip install --upgrade google-cloud-storage > log.txt
!pip install tqdm > log.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 1.6.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.22.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 2.34.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud

In [3]:
import shioaji
#import dash
#from jupyter_dash import JupyterDash
#import dash_core_components as dcc
#import dash_html_components as html
#from dash.dependencies import Output, Input, State
#import plotly.graph_objs as go
import datetime
import pandas as pd
import pandas_gbq as pd_gbq
import os
#import sqlite3
#import yfinance as yf
%load_ext google.cloud.bigquery 
%load_ext google.colab.data_table

#Bigquery認證

In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account
import googleapiclient
from googleapiclient import discovery

project_id = 'project4bigquery'
credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/MyDrive/project4bigquery_credential.json'
    #'/content/project4bigquery_credential.json'
        )

scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

client = bigquery.Client(project=project_id, credentials= credentials, )
compute = discovery.build('compute', 'v1', credentials=credentials)
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import googleapiclient
from googleapiclient import discovery

project_id = 'crypto-trading-20210130'
credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/MyDrive/crypto-trading-20210130-098eb822fa12.json')

scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

client = bigquery.Client(project=project_id, credentials= credentials)
compute = googleapiclient.discovery.build('compute', 'v1', credentials=credentials)
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


#建立Instances
https://dev.to/zohebabai/boost-your-colab-notebooks-with-gcp-and-aws-instance-within-a-few-minutes-47ma

In [ ]:
!gcloud init

In [ ]:
!gcloud auth login

In [ ]:
%env GCE_PROJECT_NAME project4bigquery
%env ACCOUNT yschan1101026@gmail.com
%env ZONE asia-east1-b
%env VM_NAME colab-1-vm
%env TPU_IP_RANGE 10.240.1.0/29
%env GCS_DATA_PATH gs://cloud-tpu-data/mnist/
%env GCS_CKPT_PATH gs://cloud-tpu-checkpoint/mnist/
    
# Automatically get bucket name from GCS paths
import os
os.environ['GCS_DATA_BUCKET'] = os.environ['GCS_DATA_PATH'][5:].split('/')[0]
os.environ['GCS_CKPT_BUCKET'] = os.environ['GCS_CKPT_PATH'][5:].split('/')[0]

env: GCE_PROJECT_NAME=project4bigquery
env: ACCOUNT=yschan1101026@gmail.com
env: ZONE=asia-east1-b
env: VM_NAME=colab-1-vm
env: TPU_IP_RANGE=10.240.1.0/29
env: GCS_DATA_PATH=gs://cloud-tpu-data/mnist/
env: GCS_CKPT_PATH=gs://cloud-tpu-checkpoint/mnist/


In [ ]:
!gcloud config set account $ACCOUNT
!gcloud services enable storage.googleapis.com --project $GCE_PROJECT_NAME
!gcloud config set compute/zone $ZONE 
#!gcloud config set core/project $GCE_PROJECT_NAME

Updated property [core/account].
Updated property [compute/zone].


In [ ]:
zone= %env ZONE
def list_instances(compute= compute, project= project_id, zone= zone):
    result = compute.instances().list(project=project, zone=zone).execute()
    return result['items'] if 'items' in result else print('No existing items')
list_instances()

No existing items


In [ ]:
#!gcloud alpha compute tpus create $TPU_NAME --range=$TPU_IP_RANGE --accelerator-type= --version=nightly --zone=$TPU_ZONE
!gcloud beta compute instances create $VM_NAME \
    --machine-type=e2-standard-4 \
    --zone=$ZONE \

Created [https://www.googleapis.com/compute/beta/projects/project4bigquery/zones/asia-east1-b/instances/colab-1-vm].
NAME        ZONE          MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
colab-1-vm  asia-east1-b  e2-standard-4               10.140.0.30  35.229.218.126  RUNNING


In [ ]:
!gcloud compute ssh --zone $ZONE $VM_NAME -- -L 8888:localhost:8888

In [ ]:
!yes | gcloud beta compute instances delete $VM_NAME

The following instances will be deleted. Any attached disks configured to be 
auto-deleted will be deleted unless they are attached to any other instances or 
the `--keep-disks` flag is given and specifies them for keeping. Deleting a disk
 is irreversible and any data on the disk will be lost.
 - [colab-1-vm] in [asia-east1-b]

Do you want to continue (Y/n)?  
Deleted [https://www.googleapis.com/compute/beta/projects/project4bigquery/zones/asia-east1-b/instances/colab-1-vm].


# Shioaji

In [ ]:
# 匯入 Shioaji 套件
import shioaji as sj

# 建立 Shioaji api 物件
api = sj.Shioaji()

# 登入帳號
api.login(
     person_id="E124281388",
     passwd="lz5014225",
     contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
     )

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.


[FutureAccount(person_id='E124281388', broker_id='F002000', account_id='1564279', username='詹詠翔'),
 Account(account_type=<AccountType.H: 'H'>, person_id='E124281388', broker_id='9A95', account_id='0184746', username='詹詠翔'),
 StockAccount(person_id='E124281388', broker_id='9A95', account_id='0631362', username='詹詠翔\u3000\u3000')]

# 列出股票及期貨清單

In [ ]:
# 檢查BigQuery現有資料表
dataset_ref = client.dataset('kbar', project= project_id)
dataset = client.get_dataset(dataset_ref)
tables =client.get_table('kbar.close_5min')
tables = list(client.list_tables(dataset))
table = [table.table_id for table in tables]
print(table, '\n')


['close_5min', 'close_day', 'high_5min', 'high_day', 'low_5min', 'low_day', 'open_5min', 'open_day', 'volume_5min', 'volume_day'] 



In [ ]:
from shioaji.constant import Exchange
import datetime
import re

contracts = api.Contracts
stock = contracts.Stocks
future = contracts.Futures

stock_list = []
for s in stock:
  for f in s:
    stock_list.append({**f})
df_stock = pd.DataFrame(stock_list).set_index('code').dropna()
#df_stock.to_gbq(destination_table= 'my_database.stock_list', project_id= project_id, if_exists='replace', location= 'asia-east1') #存在台灣的資料庫

future_list = []
for s in future:
  for f in s:
    future_list.append({**f})
df_future = pd.DataFrame(future_list).set_index('code').dropna()
#df_future.to_gbq(destination_table= 'my_database.future_list', project_id= project_id, if_exists='replace', location= 'asia-east1')

## 存入股票代號及名稱
LAST_TRADE_DATE = max(df_stock.update_date)
list_stock = []
for exchange in stock:
    for stocks in exchange:
        if stocks.exchange in (Exchange.TSE, Exchange.OTC):
          if stocks.category == '00' or stocks.category == '':
                continue
          elif stocks.update_date != LAST_TRADE_DATE: #剔除已下市股票
                continue
          elif re.search('[A-Z]', stocks.code) is None:
            list_stock.append([stocks.code, stocks.name, stocks.update_date, stocks.day_trade])

df = pd.DataFrame(list_stock, columns= ['Code', 'Name', 'update_date', 'day_trade'])
df.index = df.index + 1
df.update_date = pd.to_datetime(df.update_date)
#df.to_gbq(f'{dataset}.list_stock', project_id= project_id, if_exists= 'replace', table_schema= [{'name': 'Code', 'type':'STRING'},{'name': 'Name', 'type':'STRING'}, {'name': 'update_date', 'type':'TIMESTAMP'}, {'name': 'day_trade', 'type':'STRING'}] )
print(f'股票數量:{len(df)}')

股票數量:1745


#查詢現有資料日期及欄位

In [ ]:
from datetime import datetime as dt
from datetime import timedelta
from talib.abstract import *
import numpy as np

now = str(dt.now().date())
start="2018-12-07" 
end= now
## Get ticks all day
import pandas as pd
import time
stock = api.Contracts.Stocks

stocks_table = {'code':[], 'name':[], 'update_date':[], 'symbol':[]}
for exchange in stock:
    for stocks in exchange:
        if stocks.exchange in (Exchange.TSE, Exchange.OTC):
          if stocks.category == '00' or stocks.category == '':
                continue
#          elif stocks.update_date != max(stocks.update_date): #剔除已下市股票
#                continue
          elif re.search('[A-Z]', stocks.code) is None:
            stocks_table['code'].append(stocks.code)
            stocks_table['name'].append(stocks.name)
            stocks_table['update_date'].append(stocks.update_date)
            stocks_table['symbol'].append(stocks.symbol)

stocks_table = pd.DataFrame(stocks_table)
stocks_table.set_index('symbol', inplace= True)
LAST_TRADE_DATE = pd.to_datetime(max(stocks_table.update_date)).strftime('%Y-%m-%d')

def kbar(*args):
  for input in args:
#    for stocks in {**stock.TSE, **stock.OTC}.values():
      if input in set(stocks_table.code):
        stockcode = str(input)
        kbars = api.kbars(api.Contracts.Stocks[f'{stockcode}'], start=start)
        df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts), ).drop('ts', axis= 1)
      elif input in set(stocks_table.name):
        stockcode = stocks_table[(stocks_table.name == input)].code.values[0]
        kbars = api.kbars(api.Contracts.Stocks[f'{stockcode}'], start=start)
        df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
  return df
n= 0
#for code in stocks_table.code[0:4]:
#  print([*stocks_table[stocks_table.code == code].name][0])

In [ ]:
df_0050 = pd.DataFrame({**api.kbars(stock['0050'], start= '2018-12-07')}).set_index('ts')
df_0050.index = pd.to_datetime(df_0050.index)
list_date = df_0050.resample('D').agg(
    {
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last'}   
).dropna(how='all')
list_date = pd.to_datetime(list_date.index.date)

# 確認現有欄位是否需更新外，亦可在無資料時重新抓取資料

In [5]:
import pandas as pd
import time
import tqdm.notebook as tq
import sys
from datetime import datetime as dt
from datetime import timedelta
import numpy as np
from shioaji.constant import Exchange
import re
import shioaji as sj
from google.cloud import bigquery
from google.oauth2 import service_account
import googleapiclient
from googleapiclient import discovery

api = sj.Shioaji()

class gbq_trading:
  def __init__(self, id, passwd):
    self.start = '2018-12-07'
    self.now = str(dt.now().date())
    if not api.stock_account:
      api.login(
         person_id= id,
         passwd= passwd,
         contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
         )
      self.stock = api.Contracts.Stocks
    else:
      self.stock = api.Contracts.Stocks   
  
  def stocks_table(self):
    stocks_table = {'code':[], 'name':[], 'update_date':[], 'symbol':[]}
    for exchange in self.stock:
        for stocks in exchange:
            if stocks.exchange in (Exchange.TSE, Exchange.OTC):
              if stocks.category == '00' or stocks.category == '':
                continue
              elif re.search('[A-Z]', stocks.code) is None:
                stocks_table['code'].append(stocks.code)
                stocks_table['name'].append(stocks.name)
                stocks_table['update_date'].append(stocks.update_date)
                stocks_table['symbol'].append(stocks.symbol)
    stocks_table = pd.DataFrame(stocks_table)
    stocks_table.set_index('symbol', inplace= True)
    return stocks_table[stocks_table.update_date == max(stocks_table.update_date)]     

  def list_date(self): 
    # all kbar date data 
    df_0050 = pd.DataFrame({**api.kbars(self.stock['0050'], start= self.start)}).set_index('ts')
    df_0050.index = pd.to_datetime(df_0050.index)
    list_date = df_0050.resample('D').agg({
                'Open': 'first', 
                'High': 'max', 
                'Low': 'min', 
                'Close': 'last'}   
                ).replace(0, np.NAN).dropna(how='all')
    list_date = pd.to_datetime(list_date.index.date)
    return list_date

  def kbar_shioaji(self, args, start= '2018-12-07', end= dt.now().strftime('%Y-%m-%d')): 
    # download kbar data from shioaji 
    # instead of using *args in argumnets to allow list as input
    sotck = self.stock
    stocks_table = self.stocks_table()
    df_kbars = []
    keys = []
    for input in tq.tqdm(args, desc= 'Downloading', position= 0):
      if input in [stocks.code for stocks in {**stock.TSE, **stock.OTC}.values()]:
        keys.append(input)
        kbars = api.kbars(stock[f'{input}'], start= start, end= end)
        df_kbars.append(pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1).resample(rule='1Min', closed = 'right').agg({
                     'Open': 'first', 
                     'High': 'max',            
                     'Low': 'min',
                     'Close': 'last',
                     'Volume': 'sum'}).dropna(axis= 0, how= 'all'))
      elif input in [stocks.name for stocks in {**stock.TSE, **stock.OTC}.values()]:
        stockcode = stocks_table[stocks_table.name == input].code[0]
        keys.append(stockcode)    
        kbars = api.kbars(stock[f'{stockcode}'], start= start, end= end)
        df_kbars.append(pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1).resample(rule='1Min', closed = 'right').agg({
                     'Open': 'first', 
                     'High': 'max',            
                     'Low': 'min',
                     'Close': 'last',
                     'Volume': 'sum'}).dropna(axis= 0, how= 'all'))
      elif input not in [*stocks_table.code] + [*stocks_table.name]:
        raise KeyError(f'請確認{input}是否正確')
      dfs = pd.concat(df_kbars, axis= 1, keys= keys)    
    return dfs

  def kbar_gbq(self, args):
    # query pre-stored kbar data from Google bigquery 
    stocks_table = self.stocks_table()
    df_kbars = []
    columns = {}
    for input in args:
      if input in stocks_table.code.values:
        dfs = client.query('''
                    SELECT
                    ts,
                    {}
                    FROM
                    `{}.{}.{}`
                    ORDER BY ts ASC 
                    '''.format(stocks_table[stocks_table.code.values == input].index[0], project_id, dataset, table_id)).to_dataframe().set_index('ts')
        df_kbars.append(dfs)
        columns[dfs.columns[0]] = input
      elif input in stocks_table.name.values:
        dfs = client.query('''
                    SELECT
                    ts,
                    {}
                    FROM
                    `{}.{}.{}`
                    ORDER BY ts ASC 
                    '''.format(stocks_table[stocks_table.name.values == input].index[0], project_id, dataset, table_id)).to_dataframe().set_index('ts')
        df_kbars.append(dfs)
        columns[dfs.columns[0]] = stocks_table[stocks_table.name == input].code.values[0]

    dfs = pd.concat(df_kbars, axis= 1).rename(columns= columns)
    return dfs

  def update_to_gbq(self, project_id, dataset, credentials_path):
    self.stocks_table = self.stocks_table()
    update_date = self.stocks_table.update_date
    list_date = self.list_date()
    df_list = {}
    df_to_add_list = {}
    column_name = {}

    credentials = service_account.Credentials.from_service_account_file(credentials_path)
    scoped_credentials = credentials.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])
    client = bigquery.Client(project= project_id, credentials= credentials)

    dataset_ref = client.dataset('kbar', project= project_id)
    datasets = client.get_dataset(dataset_ref)
    tables =client.get_table('kbar.close_5min')
    tables = list(client.list_tables(datasets))
    table_list = [table.table_id for table in tables]
    
    for table_id in table_list:
      update = 0
      new = 0
      #To get full table information and allow appending collumns at first
      #查詢最新更新日期
      update_time= {} #避免保留上一輪loop的table_list資料
      last_update= {}
      init_update= {}
      for id in table_list[table_list.index(table_id):]: 
        try:
          update_time[id] = pd.to_datetime(client.query('''
                                  SELECT
                                  ts
                                  FROM
                                  `{}.{}.{}`
                                  ORDER BY ts ASC
                                  '''.format(project_id, dataset, id)).to_dataframe().ts.values).tz_localize(None)
          last_update[id] = update_time[id].max()
          init_update[id] = update_time[id].min()
        except:
          print(f'Oops! 須確認{id}的更新日期')
          update_time[id] = []
          last_update[id] = pd.to_datetime('2018-12-06') #如果沒有資料，設定成可下載期間初始日的前一日，在計算回推日期時，初始點(point_str)才是正確的
          init_update[id] = pd.to_datetime('2018-12-06')
  
      #檢查現有欄位
      column_name[f'{table_id}'] =  client.query('''
                              SELECT
                              * 
                              FROM
                              `{}.{}`.INFORMATION_SCHEMA.COLUMNS
                              WHERE table_name="{}"'''.format(project_id, dataset, table_id)).to_dataframe().column_name.values.tolist()[1:] #省略ts

      #允許新增欄位
      job_config = bigquery.QueryJobConfig(
                  destination= f"{project_id}.{dataset}.{table_id}",
                  schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION],
                  write_disposition= bigquery.WriteDisposition.WRITE_APPEND,
                  )

      #如果沒有全數table_list的資料，則逐批重新抓取
      if len(update_time[f'{table_id}'])== 0:   
        n = 0
        intv = 200
        for intv in tq.tqdm([list_date[i:i+intv] for i in range(0, len(list_date), intv)], desc= '分批重新下載', position= 0):

          date_begin= intv[0]  
          date_end = intv[-1]
          sys.stdout.write('\r' +f'下載{date_begin}到{date_end}的資料')
          n += 1
          new = 0   #下一個time loop 從頭開始計算次數

          for table in table_list:
            df_list[table]= pd.DataFrame([])

          for code in tq.tqdm(self.stocks_table.code, desc= '重新抓取資料', position= 1):
            kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= date_begin.strftime('%Y-%m-%d'), end= date_end.strftime('%Y-%m-%d'))
            df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
            df.index.rename('ts', inplace= True)
            df_5min = df.resample(rule='5Min', closed='right').agg({
                'Open': 'first', 
                'High': 'max', 
                'Low': 'min', 
                'Close': 'last',
                'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
            df_day = df.resample(rule='D').agg({
                'Open': 'first', 
                'High': 'max', 
                'Low': 'min', 
                'Close': 'last',
                'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
            df_5min.sort_index(inplace= True)
            df_day.sort_index(inplace= True)
            #5min
            #close
            df_list['close_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Close       
            #open
            df_list['open_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Open
            #high
            df_list['high_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.High
            #low
            df_list['low_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Low
            #volume
            df_list['volume_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Volume

            #day
            #close
            df_list['close_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Close
            #open
            df_list['open_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Open
            #high
            df_list['high_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.High
            #low
            df_list['low_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Low
            #volume
            df_list['volume_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Volume
            new += 1
            sys.stdout.write('\r' + f'{table_id}第{n}輪已併入{new}項，還剩下{len(self.stocks_table.index) - new + 1}項')
            sys.stdout.flush()

      ##存入資料
          for id in [id for id in table_list if len(update_time[id]) != len(list_date)]:
            df_list[id].reset_index().to_gbq(f'{dataset}.{id}', project_id= project_id, if_exists='append', credentials= credentials)
            sys.stdout.write('\r' + f'第{n}輪重新下載{id}')
            sys.stdout.flush()
      #若有現有欄位，則須確認資料是否有缺漏
      elif column_name[f'{table_id}']:

        existing_data = [dt for dt in list_date if dt in update_time[f'{table_id}'].date] #現有資料日期
        miss_data = [dt for dt in list_date if dt not in update_time[f'{table_id}'].date] #缺少資料的日期

        if len(existing_data) == len(list_date) and last_update[f'{table_id}'].date() == list_date[-1]: #若更新日期為最新且資料長度與資料來源一致，則可以判斷是最新資料
          print('\r' + f'{table_id}已是最新資料')
          continue

        elif min(miss_data) > last_update[f'{table_id}']: #如果缺失資料的最早日期晚於最後更新日期，則只要更新最後更新日期之後的資料

          point_str = -sum(list_date > last_update[f'{table_id}'])
          sys.stdout.write('\r' + f"{table_id}須更新{list_date[point_str].date()}後的資料")

          for table in table_list:
            df_list[table] = pd.DataFrame([])
            df_to_add_list[table] = pd.DataFrame([])

          for code in tq.tqdm(self.stocks_table.loc[[symbol for symbol in self.stocks_table.index if symbol in column_name[f'{table_id}']]].code, desc= '更新現有欄位資料', position= 1):
            kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= min(miss_data).strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
            df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
            df.index.rename('ts', inplace= True)
            df_5min = df.resample(rule='5Min', closed='right').agg({
                'Open': 'first', 
                'High': 'max', 
                'Low': 'min', 
                'Close': 'last',
                'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
            df_day = df.resample(rule='D').agg({
                'Open': 'first', 
                'High': 'max', 
                'Low': 'min', 
                'Close': 'last',
                'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
            df_5min.sort_index(inplace= True)
            df_day.sort_index(inplace= True)

            df_list['close_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Close
            df_list['open_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Open
            df_list['high_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.High
            df_list['low_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Low    
            df_list['volume_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Volume

            df_list['close_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Close
            df_list['open_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Open
            df_list['high_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.High
            df_list['low_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Low
            df_list['volume_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Volume
            update += 1
            sys.stdout.write('\r' + f'{table_id}還剩下{len(column_name[table_id]) - update}待更新，並須插入{len([symbols for symbols in stocks_table.index if symbols not in column_name[table_id]])}項')
            sys.stdout.flush()

        elif min(miss_data) < last_update[f'{table_id}']: #若缺失的最早資料早於最後更新日期，代表在最後更新日期前的資料有缺少

          for table in table_list:
            df_list[table] = pd.DataFrame([])
            df_to_add_list[table] = pd.DataFrame([])      

          for date in tq.tqdm([miss_data[i:i+1] for i in range(0, len(miss_data), 1)], desc= '補齊缺失資料', position= 1):
            for code in tq.tqdm(stocks_table.loc[[symbol for symbol in self.stocks_table.index if symbol in column_name[f'{table_id}']]].code, desc= '更新現有欄位資料', position= 1):
              kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= date[0].strftime('%Y-%m-%d'), end= date[0].strftime('%Y-%m-%d'))
              df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
              df.index.rename('ts', inplace= True)
              df_5min = df.resample(rule='5Min', closed='right').agg({
                  'Open': 'first', 
                  'High': 'max', 
                  'Low': 'min', 
                  'Close': 'last',
                  'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
              df_day = df.resample(rule='D').agg({
                  'Open': 'first', 
                  'High': 'max', 
                  'Low': 'min', 
                  'Close': 'last',
                  'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
              df_5min.sort_index(inplace= True)
              df_day.sort_index(inplace= True)
              #close_5min['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
              #close_5min.index.rename('ts', inplace= True)
              df_list['close_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Close
              df_list['open_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Open
              df_list['high_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.High
              df_list['low_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Low    
              df_list['volume_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min.Volume

              df_list['close_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Close
              df_list['open_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Open
              df_list['high_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.High
              df_list['low_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Low
              df_list['volume_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day.Volume
              update += 1
              sys.stdout.write('\r' + f'{table_id}還剩下{len(column_name[table_id]) - update}待更新，並須插入{len(stocks_table.index) - len(column_name[table_id])}項')
              sys.stdout.flush()

        for code in tq.tqdm(self.stocks_table.loc[[symbol for symbol in self.stocks_table.index if symbol not in column_name[f'{table_id}']]].code, desc= '新增欄位', position= 1):
          kbars_to_add = api.kbars(api.Contracts.Stocks[f'{code}'], start= list_date[0].strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
          df_to_add = pd.DataFrame({**kbars_to_add}, index= pd.to_datetime(kbars_to_add.ts)).drop('ts', axis= 1)
          df_to_add.index.rename('ts', inplace= True)
          df_5min_to_add = df_to_add.resample(rule='5Min', closed='right').agg({
              'Open': 'first', 
              'High': 'max', 
              'Low': 'min', 
              'Close': 'last',
              'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
          df_day_to_add = df_to_add.resample(rule='D').agg({
              'Open': 'first', 
              'High': 'max', 
              'Low': 'min', 
              'Close': 'last',
              'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
          df_5min_to_add.sort_index(inplace= True)
          df_day_to_add.sort_index(inplace= True)
          df_to_add_list['close_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min_to_add.Close
          df_to_add_list['open_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min_to_add.Open
          df_to_add_list['high_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min_to_add.High
          df_to_add_list['low_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min_to_add.Low    
          df_to_add_list['volume_5min']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_5min_to_add.Volume
          df_to_add_list['close_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day_to_add.Close
          df_to_add_list['open_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day_to_add.Open
          df_to_add_list['high_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day_to_add.High
          df_to_add_list['low_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day_to_add.Low
          df_to_add_list['volume_day']['{}'.format(self.stocks_table[self.stocks_table.code == code].index[0])] = df_day_to_add.Volume
          new += 1
          sys.stdout.write('\r' + f'{table_id}已插入{new}項，還剩下{len(self.stocks_table.index) - len(column_name[table_id]) - new}項')
          sys.stdout.flush()
      #存入資料
      for id in [id for id in last_update.keys() if last_update[id].date() != list_date[-1]]:
        if id:
          df_list[id].reset_index().to_gbq(f'{dataset}.{id}', project_id= project_id, if_exists='append', credentials= credentials)

      if not [symbol for symbol in stocks_table.index if symbol not in column_name]:
        table = client.get_table(f'{dataset}.{table_id}')  # Make an API request.

        original_schema = table.schema
        new_schema = original_schema[:]  # Creates a copy of the schema.
        for new_columns in [*df_to_add[f'{table_id}'].columns]:
          new_schema.append(bigquery.SchemaField(f'{new_columns}', 'FLOAT'))
        table.schema = new_schema
        table = client.update_table(table, ["schema"])  # Make an API request.

        if len(table.schema) == len(original_schema) + 1 == len(new_schema):
          print("A new column has been added.")
        else:
          print("The column has not been added.")
          # Start the query, passing in the extra configuration.
        client.load_table_from_dataframe(df_to_add[f'{table_id}'], f'{dataset}.{table_id}')
    print('Done') 

###TEST

In [ ]:
#table_list = ['close_5min', 'open_5min', 'high_5min', 'low_5min', 'volume_5min']#, 'close_day', 'open_day', 'high_day', 'low_day', 'volume_day']
#update_time = {}
#dataset= 'kbar'
#for table_id in table_list:
#  update_time[f'{table_id}'] = pd.to_datetime(client.query('''
#                          SELECT
#                          ts
#                          FROM
#                          `{}.{}.{}`
#                          ORDER BY ts ASC
#                          '''.format(project_id, dataset, table_id)).to_dataframe().ts.values)#.tz_localize(None)
##update_time['close_5min'][-sum(update_time['close_5min'] < list_date[-1])].date()
#miss = [dt for dt in list_date if dt in update_time['close_5min'].date]#.to_period('D').groupby(update_time['close_5min'].to_period('D')).keys()]
#intv = 2
#for date in [miss[i:i+intv] for i in range(0, len(miss), intv)]:
#  print(date[0].strftime('%Y-%m-%d'))
#D = update_time['close_5min'].to_period('D')
#len(D.groupby(D).keys())
#min(miss) > last_update['close_5min']
#update_time['close_5min'].date
#point_str = -(sum(update_time['close_5min'] < list_date[-1]))
#update_time['close_5min'][point_str].strftime('%Y-%m-%d')
#init_update['close_5min']
#min(miss).strftime('%Y-%m-%d')
#update_time[f'{table_id}'].date
#table_id= 'close_5min'
#df = client.query("""
#        SELECT
#        ts,
#        OTC8034, 
#        OTC5487, 
#        OTC5386, 
#        OTC2745,
#        FROM
#        `{}.{}.{}`
#        ORDER BY
#        ts
#        """.format(project_id, dataset, table_id)).to_dataframe().set_index('ts')
#df = df['2018-12-07':'2019-06-30'].iloc[0:-1:48].append(df['2019-07-01': '2019-12-31'].iloc[0:-1:72])
#update_date = pd.to_datetime(df.index.values)
#miss_data = [dt for dt in list_date if dt not in update_date.date]
#
#for date in [miss_data[i:i+2] for i in range(0, len(miss_data), 2)]:
#list_date.strftime('%Y-%m-%d').to_list().index('2021-12-17')  
#miss_data
#for date in [miss_data[i:i+2] for i in range(0, len(miss_data), 2)if (list_date.to_list().index(miss_data[i]) - list_date.to_list().index(miss_data[i -1]) != 1) | (list_date.to_list().index(miss_data[i]) - list_date.to_list().index(miss_data[i -1]) != -1) ]:
#  print(date)

# 關閉虛擬主機

In [ ]:
!yes | gcloud beta compute instances delete $VM_NAME

#以前版本

###從Shioaji抓資料及查詢Bigquery資料

In [ ]:
stock = api.Contracts.Stocks
def kbar_gbq(args):
  df_kbars = []
  columns = {}
  for input in args:
    if input in stocks_table.code.values:
      dfs = client.query('''
                  SELECT
                  ts,
                  {}
                  FROM
                  `{}.{}.{}`
                  ORDER BY ts ASC 
                  '''.format(stocks_table[stocks_table.code.values == input].index[0], project_id, dataset, table_id)).to_dataframe().set_index('ts')
      df_kbars.append(dfs)
      columns[dfs.columns[0]] = input
    elif input in stocks_table.name.values:
      dfs = client.query('''
                  SELECT
                  ts,
                  {}
                  FROM
                  `{}.{}.{}`
                  ORDER BY ts ASC 
                  '''.format(stocks_table[stocks_table.name.values == input].index[0], project_id, dataset, table_id)).to_dataframe().set_index('ts')
      df_kbars.append(dfs)
      columns[dfs.columns[0]] = stocks_table[stocks_table.name == '台積電'].code.values[0]

  dfs = pd.concat(df_kbars, axis= 1).rename(columns= columns)
  return dfs
#dfs.rename(columns= {dfs.columns[0]: keys})

def kbar_shioaji(args, start= '2018-12-07', end= dt.now().strftime('%Y-%m-%d')): 
  # download kbar data from shioaji 
  # instead of using *args in argumnets to allow list as input
  df_kbars = []
  keys = []
  for input in tq.tqdm(args, desc= 'Downloading', position= 0):
    if input in [stocks.code for stocks in {**stock.TSE, **stock.OTC}.values()]:
      keys.append(input)
      kbars = api.kbars(stock[f'{input}'], start= start, end= end)
      df_kbars.append(pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1).resample(rule='1Min', closed = 'right').agg({
                   'Open': 'first', 
                   'High': 'max',            
                   'Low': 'min',
                   'Close': 'last',
                   'Volume': 'sum'}).dropna(axis= 0, how= 'all'))
    elif input in [stocks.name for stocks in {**stock.TSE, **stock.OTC}.values()]:
      stockcode = stocks_table[stocks_table.name == input].code[0]
      keys.append(stockcode)    
      kbars = api.kbars(stock[f'{stockcode}'], start= start, end= end)
      df_kbars.append(pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1).resample(rule='1Min', closed = 'right').agg({
                   'Open': 'first', 
                   'High': 'max',            
                   'Low': 'min',
                   'Close': 'last',
                   'Volume': 'sum'}).dropna(axis= 0, how= 'all'))
    elif input not in [*stocks_table.code] + [*stocks_table.name]:
      raise KeyError(f'請確認{input}是否正確')
    dfs = pd.concat(df_kbars, axis= 1, keys= keys)    
  return dfs

###ver.4

In [ ]:
import pandas as pd
import time
import tqdm.notebook as tq
import sys
from datetime import datetime as dt
from datetime import timedelta
import numpy as np
from shioaji.constant import Exchange
import re
import shioaji as sj
from google.cloud import bigquery
from google.oauth2 import service_account
import googleapiclient
from googleapiclient import discovery

start="2018-12-07" 
end = str(dt.now().date())
dataset = 'kbar'

stocks_table = {'code':[], 'name':[], 'update_date':[], 'symbol':[]}
for exchange in stock:
    for stocks in exchange:
        if stocks.exchange in (Exchange.TSE, Exchange.OTC):
          if stocks.category == '00' or stocks.category == '':
                continue
          elif re.search('[A-Z]', stocks.code) is None:
            stocks_table['code'].append(stocks.code)
            stocks_table['name'].append(stocks.name)
            stocks_table['update_date'].append(stocks.update_date)
            stocks_table['symbol'].append(stocks.symbol)
stocks_table = pd.DataFrame(stocks_table)
stocks_table.set_index('symbol', inplace= True)
LAST_TRADE_DATE = max(stocks_table.update_date)
stocks_table = stocks_table[stocks_table.update_date == LAST_TRADE_DATE]

def kbar(*args):
  for input in args:
    for stocks in {**stock.TSE, **stock.OTC}.values():
      if input in set(stocks_table.code):
        stockcode = str(input)
        kbars = api.kbars(api.Contracts.Stocks[f'{stockcode}'], start=start)
        df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts), ).drop('ts', axis= 1)
      elif input in set(stocks_table.name):
        stockcode = stocks_table[(stocks_table.name == input)].code.values[0]
        kbars = api.kbars(api.Contracts.Stocks[f'{stockcode}'], start=start)
        df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
  return df


#透過0050取得完整資料序列
df_0050 = pd.DataFrame({**api.kbars(stock['0050'], start= '2018-12-07')}).set_index('ts')
df_0050.index = pd.to_datetime(df_0050.index)
list_date = df_0050.resample('D').agg({
            'Open': 'first', 
            'High': 'max', 
            'Low': 'min', 
            'Close': 'last'}   
            ).dropna(how='all')
list_date = pd.to_datetime(list_date.index.date)

#取得現有資料表
dataset_ref = client.dataset('kbar', project= project_id)
dataset = client.get_dataset(dataset_ref)
tables =client.get_table('kbar.close_5min')
tables = list(client.list_tables(dataset))
table_list = [table.table_id for table in tables]

#更新資料
dataset= 'kbar'
#table_list = ['close_5min', 'open_5min', 'high_5min', 'low_5min', 'volume_5min', 'close_day', 'open_day', 'high_day', 'low_day', 'volume_day'] 
update_date = stocks_table.update_date
df_list = {}
df_to_add_list = {}
column_name = {}

for table_id in table_list:
  update = 0
  new = 0
  #To get full table information and allow appending collumns at first
  #查詢最新更新日期
  update_time= {} #避免保留上一輪loop的table_list資料
  last_update= {}
  init_update= {}
  for id in table_list[table_list.index(table_id):]: 
    try:
      update_time[id] = pd.to_datetime(client.query('''
                              SELECT
                              ts
                              FROM
                              `{}.{}.{}`
                              ORDER BY ts ASC
                              '''.format(project_id, dataset, id)).to_dataframe().ts.values).tz_localize(None)
      last_update[id] = update_time[id].max()
      init_update[id] = update_time[id].min()
    except:
      print(f'Oops! 須確認{id}的更新日期')
      update_time[id] = []
      last_update[id] = pd.to_datetime('2018-12-06') #如果沒有資料，設定成可下載期間初始日的前一日，在計算回推日期時，初始點(point_str)才是正確的
      init_update[id] = pd.to_datetime('2018-12-06')
  
  #檢查現有欄位
  column_name[f'{table_id}'] =  client.query('''
                          SELECT
                          * 
                          FROM
                          `{}.{}`.INFORMATION_SCHEMA.COLUMNS
                          WHERE table_name="{}"'''.format(project_id, dataset, table_id)).to_dataframe().column_name.values.tolist()[1:] #省略ts

  #允許新增欄位
  job_config = bigquery.QueryJobConfig(
              destination= f"{project_id}.{dataset}.{table_id}",
              schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION],
              write_disposition= bigquery.WriteDisposition.WRITE_APPEND,
              )

  #如果沒有全數table_list的資料，則逐批重新抓取
  if len(update_time[f'{table_id}'])== 0:   
    n = 0
    intv = 200
    for intv in tq.tqdm([list_date[i:i+intv] for i in range(0, len(list_date), intv)], desc= '分批重新下載', position= 0):
      
      date_begin= intv[0]  
      date_end = intv[-1]
      sys.stdout.write('\r' +f'下載{date_begin}到{date_end}的資料')
      n += 1
      new = 0   #下一個time loop 從頭開始計算次數

      for table in table_list:
        df_list[table]= pd.DataFrame([])

      for code in tq.tqdm(stocks_table.code, desc= '重新抓取資料', position= 1):
        kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= date_begin.strftime('%Y-%m-%d'), end= date_end.strftime('%Y-%m-%d'))
        df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
        df.index.rename('ts', inplace= True)
        df_5min = df.resample(rule='5Min', closed='right').agg({
            'Open': 'first', 
            'High': 'max', 
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
        df_day = df.resample(rule='D').agg({
            'Open': 'first', 
            'High': 'max', 
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
        df_5min.sort_index(inplace= True)
        df_day.sort_index(inplace= True)
        #5min
        #close
        df_list['close_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close       
        #open
        df_list['open_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Open
        #high
        df_list['high_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.High
        #low
        df_list['low_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Low
        #volume
        df_list['volume_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Volume

        #day
        #close
        df_list['close_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Close
        #open
        df_list['open_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Open
        #high
        df_list['high_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.High
        #low
        df_list['low_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Low
        #volume
        df_list['volume_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Volume
        new += 1
        sys.stdout.write('\r' + f'{table_id}第{n}輪已併入{new}項，還剩下{len(stocks_table.index) - new + 1}項')
        sys.stdout.flush()

  ##存入資料
      for id in [id for id in table_list if len(update_time[id]) != len(list_date)]:
        df_list[id].reset_index().to_gbq(f'{dataset}.{id}', project_id= project_id, if_exists='append', credentials= credentials)
        sys.stdout.write('\r' + f'第{n}輪重新下載{id}')
        sys.stdout.flush()
      #df_list['close_5min'].reset_index().to_gbq(f'{dataset}.close_5min', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['open_5min'].reset_index().to_gbq(f'{dataset}.open_5min', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['high_5min'].reset_index().to_gbq(f'{dataset}.high_5min', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['low_5min'].reset_index().to_gbq(f'{dataset}.low_5min', project_id= project_id, if_exists='replace')
      #df_list['volume_5min'].reset_index().to_gbq(f'{dataset}.volume_5min', project_id= project_id, if_exists='replace', credentials= credentials)

      #df_list['close_day'].reset_index().to_gbq(f'{dataset}.close_day', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['open_day'].reset_index().to_gbq(f'{dataset}.open_day', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['high_day'].reset_index().to_gbq(f'{dataset}.high_day', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['low_day'].reset_index().to_gbq(f'{dataset}.low_day', project_id= project_id, if_exists='replace', credentials= credentials)
      #df_list['volume_day'].reset_index().to_gbq(f'{dataset}.volume_day', project_id= project_id, if_exists='replace', credentials= credentials)   
  
  #若有現有欄位，則須確認資料是否有缺漏
  elif column_name[f'{table_id}']:
    
    existing_data = [dt for dt in list_date if dt in update_time[f'{table_id}'].date] #現有資料日期
    miss_data = [dt for dt in list_date if dt not in update_time[f'{table_id}'].date] #缺少資料的日期

    if len(existing_data) == len(list_date) and last_update[f'{table_id}'].date() == list_date[-1]: #若更新日期為最新且資料長度與資料來源一致，則可以判斷是最新資料
      print(f'{table_id}已是最新資料')
      continue

    elif min(miss_data) > last_update[f'{table_id}']: #如果缺失資料的最早日期晚於最後更新日期，則只要更新最後更新日期之後的資料

      point_str = -sum(list_date > last_update[f'{table_id}'])
      sys.stdout.write('\r' + f"{table_id}須更新{list_date[point_str].date()}後的資料")

      for table in table_list:
        df_list[table] = pd.DataFrame([])
        df_to_add_list[table] = pd.DataFrame([])

      for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol in column_name[f'{table_id}']]].code, desc= '更新現有欄位資料', position= 1):
        kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= min(miss_data).strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
        df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
        df.index.rename('ts', inplace= True)
        df_5min = df.resample(rule='5Min', closed='right').agg({
            'Open': 'first', 
            'High': 'max', 
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
        df_day = df.resample(rule='D').agg({
            'Open': 'first', 
            'High': 'max', 
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
        df_5min.sort_index(inplace= True)
        df_day.sort_index(inplace= True)
        #close_5min['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
        #close_5min.index.rename('ts', inplace= True)
        df_list['close_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
        df_list['open_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Open
        df_list['high_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.High
        df_list['low_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Low    
        df_list['volume_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Volume

        df_list['close_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Close
        df_list['open_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Open
        df_list['high_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.High
        df_list['low_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Low
        df_list['volume_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Volume
        update += 1
        sys.stdout.write('\r' + f'{table_id}還剩下{len(column_name[table_id]) - update}待更新，並須插入{len([symbols for symbols in stocks_table.index if symbols not in column_name[table_id]])}項')
        sys.stdout.flush()
    
    elif min(miss_data) < last_update[f'{table_id}']: #若缺失的最早資料早於最後更新日期，代表在最後更新日期前的資料有缺少

      for table in table_list:
        df_list[table] = pd.DataFrame([])
        df_to_add_list[table] = pd.DataFrame([])      
      
      for date in tq.tqdm([miss_data[i:i+1] for i in range(0, len(miss_data), 1)], desc= '補齊缺失資料', position= 1):
        for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol in column_name[f'{table_id}']]].code, desc= '更新現有欄位資料', position= 1):
          kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= date[0].strftime('%Y-%m-%d'), end= date[0].strftime('%Y-%m-%d'))
          df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
          df.index.rename('ts', inplace= True)
          df_5min = df.resample(rule='5Min', closed='right').agg({
              'Open': 'first', 
              'High': 'max', 
              'Low': 'min', 
              'Close': 'last',
              'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
          df_day = df.resample(rule='D').agg({
              'Open': 'first', 
              'High': 'max', 
              'Low': 'min', 
              'Close': 'last',
              'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
          df_5min.sort_index(inplace= True)
          df_day.sort_index(inplace= True)
          #close_5min['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
          #close_5min.index.rename('ts', inplace= True)
          df_list['close_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
          df_list['open_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Open
          df_list['high_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.High
          df_list['low_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Low    
          df_list['volume_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Volume

          df_list['close_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Close
          df_list['open_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Open
          df_list['high_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.High
          df_list['low_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Low
          df_list['volume_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day.Volume
          update += 1
          sys.stdout.write('\r' + f'{table_id}還剩下{len(column_name[table_id]) - update}待更新，並須插入{len(stocks_table.index) - len(column_name[table_id])}項')
          sys.stdout.flush()

    for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol not in column_name[f'{table_id}']]].code, desc= '新增欄位', position= 1):
      kbars_to_add = api.kbars(api.Contracts.Stocks[f'{code}'], start= list_date[0].strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
      df_to_add = pd.DataFrame({**kbars_to_add}, index= pd.to_datetime(kbars_to_add.ts)).drop('ts', axis= 1)
      df_to_add.index.rename('ts', inplace= True)
      df_5min_to_add = df_to_add.resample(rule='5Min', closed='right').agg({
          'Open': 'first', 
          'High': 'max', 
          'Low': 'min', 
          'Close': 'last',
          'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
      df_day_to_add = df_to_add.resample(rule='D').agg({
          'Open': 'first', 
          'High': 'max', 
          'Low': 'min', 
          'Close': 'last',
          'Volume': 'sum'}).replace(0, np.NaN).dropna(axis= 0, how= 'all')
      df_5min_to_add.sort_index(inplace= True)
      df_day_to_add.sort_index(inplace= True)
      df_to_add_list['close_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.Close
      df_to_add_list['open_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.Open
      df_to_add_list['high_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.High
      df_to_add_list['low_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.Low    
      df_to_add_list['volume_5min']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.Volume
      df_to_add_list['close_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day_to_add.Close
      df_to_add_list['open_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day_to_add.Open
      df_to_add_list['high_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day_to_add.High
      df_to_add_list['low_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day_to_add.Low
      df_to_add_list['volume_day']['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_day_to_add.Volume
      new += 1
      sys.stdout.write('\r' + f'{table_id}已插入{new}項，還剩下{len(stocks_table.index) - len(column_name[table_id]) - new}項')
      sys.stdout.flush()
  #存入資料
  for id in [id for id in last_update.keys() if last_update[id].date() != list_date[-1]]:
    if id:
      client.load_table_from_dataframe(df_list[f'{table_id}'], f'{dataset}.{table_id}')
      #df_list[id].reset_index().to_gbq(f'{dataset}.{id}', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['close_5min'].reset_index().to_gbq(f'{dataset}.close_5min', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['open_5min'].reset_index().to_gbq(f'{dataset}.open_5min', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['high_5min'].reset_index().to_gbq(f'{dataset}.high_5min', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['low_5min'].reset_index().to_gbq(f'{dataset}.low_5min', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['volume_5min'].reset_index().to_gbq(f'{dataset}.volume_5min', project_id= project_id, if_exists='append', credentials= credentials)

  #df_list['close_day'].reset_index().to_gbq(f'{dataset}.close_day', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['open_day'].reset_index().to_gbq(f'{dataset}.open_day', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['high_day'].reset_index().to_gbq(f'{dataset}.high_day', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['low_day'].reset_index().to_gbq(f'{dataset}.low_day', project_id= project_id, if_exists='append', credentials= credentials)
  #df_list['volume_day'].reset_index().to_gbq(f'{dataset}.volume_day', project_id= project_id, if_exists='append', credentials= credentials)

  if not [symbol for symbol in stocks_table.index if symbol not in column_name]:
    table = client.get_table(f'{dataset}.{table_id}')  # Make an API request.

    original_schema = table.schema
    new_schema = original_schema[:]  # Creates a copy of the schema.
    for new_columns in [*df_to_add[f'{table_id}'].columns]:
      new_schema.append(bigquery.SchemaField(f'{new_columns}', 'FLOAT'))
    table.schema = new_schema
    table = client.update_table(table, ["schema"])  # Make an API request.

    if len(table.schema) == len(original_schema) + 1 == len(new_schema):
      print("A new column has been added.")
    else:
      print("The column has not been added.")
      # Start the query, passing in the extra configuration.
    client.load_table_from_dataframe(df_to_add[f'{table_id}'], f'{dataset}.{table_id}')

print('Done')

###Ver.3(只確認現有欄位是否須更新)

In [ ]:
from IPython.display import display
n= 0
new = 0
dataset= 'kbar'
table_id = ['close_5min', 'open_5min', 'high_5min', 'low_5min', 'volume_5min', 'close_day', 'open_day', 'high_day', 'low_day', 'volume_day'] 
#查詢現有欄位
try:
  column_name= client.query(f'''
                SELECT
                * 
                FROM
                `{project_id}`.kbar.INFORMATION_SCHEMA.COLUMNS
                WHERE table_name= {table_id}''').to_dataframe().column_name.values.tolist()
except:
  column_name = []

#允許新增欄位
job_config = bigquery.QueryJobConfig(
            destination= f"{project_id}.{dataset}.{table_id}",
            schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION],
            write_disposition= bigquery.WriteDisposition.WRITE_APPEND,
            )

#查詢最新更新日期
try:
  update_time = pd.read_gbq(f'''
                SELECT
                #min(ts),
                max(ts)
                FROM
                `{project_id}.{dataset}.{table_id}`
                ''')
  last_update = pd.to_datetime(update_time.f1_.values[0])#.strftime('%Y-%m-%d')
  init_update = pd.to_datetime(update_time.f0_.values[0])
except:
  last_update = pd.to_datetime('2018-12-06') #如果沒有資料，設定成可下載期間初始日的前一日，在計算回推日期時，初始點(point_str)才是正確的
  #init_update = pd.to_datetime('2018-12-06')

if not column_name:      #如果沒有現有欄位，則逐批重新抓取
  for date_begin, date_end in zip(list_date[0::200], list_date[199::200]):
    new += 1

    close_5min = pd.DataFrame([])
    open_5min = pd.DataFrame([])
    high_5min = pd.DataFrame([])
    low_5min = pd.DataFrame([])
    volume_5min = pd.DataFrame([])
    close_day = pd.DataFrame([])
    open_day = pd.DataFrame([])
    high_day = pd.DataFrame([])
    low_day = pd.DataFrame([])
    volume_day = pd.DataFrame([])
  
    for code in tq.tqdm(stocks_table.code, desc= 'code loop', position= 0):
      kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= date_begin.strftime('%Y-%m-%d'), end= date_end.strftime('%Y-%m-%d'))
      df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
      df_5min = df.resample(rule='5Min', closed='right').agg({
          'Open': 'first', 
          'High': 'max', 
          'Low': 'min', 
          'Close': 'last',
          'Volume': 'sum'}).replace(0, np.NaN).dropna(how= 'all')
      #df_day = df.resample(rule='D').agg({
      #    'Open': 'first', 
      #    'High': 'max', 
      #    'Low': 'min', 
      #    'Close': 'last',
      #    'Volume': 'sum'}).replace(0, np.NaN).dropna(how= 'all')

      df_5min.sort_index(inplace= True)
      #df_day.sort_index(inplace= True)
      #5min
      #close
      close_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Close
      close_5min.index.rename('ts', inplace= True)
      #open
      open_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Open
      open_5min.index.rename('ts', inplace= True)
      #high
      high_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.High
      high_5min.index.rename('ts', inplace= True)
      #low
      low_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Low
      low_5min.index.rename('ts', inplace= True)    
      #volume
      volume_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Volume
      volume_5min.index.rename('ts', inplace= True)

      #day
      #close
      close_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Close
      close_day.index.rename('ts', inplace= True)
      #open
      open_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Open
      open_day.index.rename('ts', inplace= True)
      #high
      high_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.High
      high_day.index.rename('ts', inplace= True)
      #low
      low_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Low
      low_day.index.rename('ts', inplace= True)
      #volume
      volume_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Volume
      volume_day.index.rename('ts', inplace= True)
else:
  if last_update.date() == list_date[-1]:
    sys.stdout.write('已是最新資料')
    sys.stdout.flush()
  else:
    point_str = -(sum(last_update < list_date))
    sys.stdout.write('\r' + f"須更新{list_date[point_str]}後的資料")
    
    for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol in column_name]].code, desc= 'existing column updating', position= 1):
      kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= list_date[point_str].strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
      df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
      df_5min = df.resample(rule='5Min', closed='right').agg({
          'Open': 'first', 
          'High': 'max', 
          'Low': 'min', 
          'Close': 'last',
          'Volume': 'sum'}).replace(0, np.NaN).dropna(how= 'all')
      df_5min.sort_index(inplace= True)
      close_5min['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
      close_5min.index.rename('ts', inplace= True)
      sys.stdout.write('\r' + f'還剩下{len(column_name) - n}待更新，並須插入{len(stocks_table.index) - len(column_name) - new}項')
      sys.stdout.flush()
    for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol not in column_name]].code, desc= 'new column', position= 1):
      kbars_to_add = api.kbars(api.Contracts.Stocks[f'{code}'], start= list_date[0].strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
      df_to_add = pd.DataFrame({**kbars_to_add}, index= pd.to_datetime(kbars_to_add.ts)).drop('ts', axis= 1)
      df_5min_to_add = df_to_add.resample(rule='5Min', closed='right').agg({
          'Open': 'first', 
          'High': 'max', 
          'Low': 'min', 
          'Close': 'last',
          'Volume': 'sum'}).replace(0, np.NaN)#.dropna(how= 'all')
      df_5min_to_add.sort_index(inplace= True)
      close_5min_to_add['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.Close
      close_5min_to_add.index.rename('ts', inplace= True)
      new += 1
      sys.stdout.write('\r' + f'已插入{new}項，還剩下{len(stocks_table.index) - len(column_name) - new}項')
      sys.stdout.flush()
    
if not close_5min.empty:
  close_5min.reset_index().to_gbq(f'{dataset}.close_5min', project_id= project_id, if_exists='append')
  open_5min.reset_index().to_gbq(f'{dataset}.open_5min', project_id= project_id, if_exists='append')
  high_5min.reset_index().to_gbq(f'{dataset}.high_5min', project_id= project_id, if_exists='append')
  low_5min.reset_index().to_gbq(f'{dataset}.low_5min', project_id= project_id, if_exists='append')
  volume_5min.reset_index().to_gbq(f'{dataset}.volume_5min', project_id= project_id, if_exists='append')

  close_day.reset_index().to_gbq(f'{dataset}_day.close', project_id= project_id, if_exists='append')
  open_day.reset_index().to_gbq(f'{dataset}_day.open', project_id= project_id, if_exists='append')
  high_day.reset_index().to_gbq(f'{dataset}_day.high', project_id= project_id, if_exists='append')
  low_day.reset_index().to_gbq(f'{dataset}_day.low', project_id= project_id, if_exists='append')
  volume_day.reset_index().to_gbq(f'{dataset}_day.volume', project_id= project_id, if_exists='append')

if not close_5min_to_add.empty:
    table = client.get_table(f'{dataset}.close_5min')  # Make an API request.

    original_schema = table.schema
    new_schema = original_schema[:]  # Creates a copy of the schema.
    for new_columns in [*close_5min_to_add.columns]:
      new_schema.append(bigquery.SchemaField(f'{new_columns}', 'FLOAT'))
    #for new_columns in close_5min_to_add.columns:
    #  client.query(f'''
    #          INSERT 
    #          `{dataset}.{table_id}` ({new_columns})
    #          VALUES
    #          UNNEST({[close_5min_to_add['{new_columns}'].values]})
    #          WHERE ts = {close_5min_to_add['{new_columns}'].ts}
    #        ''')

    table.schema = new_schema
    table = client.update_table(table, ["schema"])  # Make an API request.

    if len(table.schema) == len(original_schema) + 1 == len(new_schema):
      print("A new column has been added.")
    else:
      print("The column has not been added.")
      # Start the query, passing in the extra configuration.
    client.load_table_from_dataframe(close_5min_to_add, f'{dataset}.close_5min')

    print('已完成{}項'.format(new))
print('Done')

###Ver.2

In [ ]:
import tqdm.notebook as tq
import sys
n= 0
new= 0

#允許新增欄位
job_config = bigquery.QueryJobConfig(
    destination= f"{project_id}.kbar.close_5min",
    schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION],
    write_disposition= bigquery.WriteDisposition.WRITE_APPEND,
    )
#查詢最新更新日期
try:
  update_time = pd.read_gbq(f'''
                SELECT
                min(ts),
                max(ts)
                FROM
                `{project_id}.kbar.close_5min`
                ''')
  last_update = pd.to_datetime(update_time.f1_.values[0])#.strftime('%Y-%m-%d')
  init_update = pd.to_datetime(update_time.f0_.values[0])
except:
  last_update = pd.to_datetime('2018-12-06') #如果沒有資料，設定成可下載期間初始日的前一日，在計算回推日期時，初始點(point_str)才是正確的
  init_update = pd.to_datetime('2018-12-06')
##
close_5min = pd.DataFrame([])
close_5min_to_add = pd.DataFrame([])
for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol in column_name]].code):
  if last_update.date() == list_date[-1]:
    print('已是最新資料')
    break
  else:
    n += 1
    point_str = -(sum(last_update < list_date))
    sys.stdout.write('\r' + f"須更新{list_date[point_str]}後的資料", end = "\r")
    kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= list_date[point_str].strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
    df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
    df_5min = df.resample(rule='5Min', closed='right').agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'}).replace(0, np.NaN).dropna(how= 'all')
    df_5min.sort_index(inplace= True)
    close_5min['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min.Close
    close_5min.index.rename('ts', inplace= True)
    sys.stdout.write('\r' + f'還剩下{len(column_name) - n}待更新，並須插入{len(stocks_table.index) - len(column_name) - new}項')
    sys.stdout.flush()
if [symbol for symbol in stocks_table.index if symbol not in column_name]:
  for code in tq.tqdm(stocks_table.loc[[symbol for symbol in stocks_table.index if symbol not in column_name]].code):
    kbars_to_add = api.kbars(api.Contracts.Stocks[f'{code}'], start= list_date[0].strftime('%Y-%m-%d'), end= list_date[-1].strftime('%Y-%m-%d'))
    df_to_add = pd.DataFrame({**kbars_to_add}, index= pd.to_datetime(kbars_to_add.ts)).drop('ts', axis= 1)
    df_5min_to_add = df_to_add.resample(rule='5Min', closed='right').agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'}).replace(0, np.NaN)#.dropna(how= 'all')
    df_5min_to_add.sort_index(inplace= True)
    close_5min_to_add['{}'.format(stocks_table[stocks_table.code == code].index[0])] = df_5min_to_add.Close
    close_5min_to_add.index.rename('ts', inplace= True)
    sys.stdout.write('\r' + f'已須插入{new}項，還剩下{len(stocks_table.index) - len(column_name) - new}項')
    new += 1
    sys.stdout.flush()
if not close_5min.empty:
  pd_gbq.to_gbq(close_5min.reset_index(), f'{dataset}.close_5min', if_exists= 'append')

if not close_5min_to_add.empty:
  table = client.get_table(f'{dataset}.close_5min')  # Make an API request.

  original_schema = table.schema
  new_schema = original_schema[:]  # Creates a copy of the schema.
  #for new_columns in [*close_5min_to_add.columns]:
  #  new_schema.append(bigquery.SchemaField(f'{new_columns}', 'FLOAT'))
  #for new_columns in close_5min_to_add.columns:
    #  client.query(f'''
    #          INSERT 
    #          `{dataset}.{table_id}` ({new_columns})
    #          VALUES
    #          UNNEST({[close_5min_to_add['{new_columns}'].values]})
    #          WHERE ts = {close_5min_to_add['{new_columns}'].ts}
    #        ''')

  #table.schema = new_schema
  #table = client.update_table(table, ["schema"])  # Make an API request.

  #if len(table.schema) == len(original_schema) + 1 == len(new_schema):
  #  print("A new column has been added.")
  #else:
  #  print("The column has not been added.")
    # Start the query, passing in the extra configuration.
  client.load_table_from_dataframe(close_5min_to_add, f'{dataset}.close_5min')

print('done')

### Ver.1(完全從頭開始抓資料)


In [ ]:
from IPython.display import display
n= 0

for date_begin, date_end in zip(list_date[1::20], list_date[20::20]):
  if date_begin == pd.to_datetime('2018-12-10'):
    date_begin = pd.to_datetime('2018-12-07')
  else:
    pass
  n += 1
  close_5min = pd.DataFrame([])
#  open_5min = pd.DataFrame([])
#  high_5min = pd.DataFrame([])
#  low_5min = pd.DataFrame([])
#  volume_5min = pd.DataFrame([])
#  close_day = pd.DataFrame([])
#  open_day = pd.DataFrame([])
#  high_day = pd.DataFrame([])
#  low_day = pd.DataFrame([])
#  volume_day = pd.DataFrame([])
#  close_week = pd.DataFrame([])
#  open_week = pd.DataFrame([])
#  high_week = pd.DataFrame([])
#  low_week = pd.DataFrame([])
#  volume_week = pd.DataFrame([])

  for code in stocks_table.code:
    kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start= date_begin.strftime('%Y-%m-%d'), end= date_end.strftime('%Y-%m-%d'))
    df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
    df_5min = df.resample(rule='5Min', closed='right').agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'}).replace(0, np.NaN).dropna(how= 'all')
 #   df_day = df.resample(rule='D').agg({
 #         'Open': 'first', 
 #         'High': 'max', 
 #         'Low': 'min', 
 #         'Close': 'last',
 #         'Volume': 'sum'}).replace(0, np.NaN).dropna(how= 'all')

    df_5min.sort_index(inplace= True)
 #   df_day.sort_index(inplace= True)
#    #5min
#    #close
    close_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Close
    close_5min.index.rename('ts', inplace= True)
#    #open
#    open_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Open
#    open_5min.index.rename('ts', inplace= True)
#    #high
#    high_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.High
#    high_5min.index.rename('ts', inplace= True)
#    #low
#    low_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Low
#    low_5min.index.rename('ts', inplace= True)    
#    #volume
#    volume_5min['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_5min.Volume
#    volume_5min.index.rename('ts', inplace= True)
#
#    #day
#    #close
#    close_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Close
#    close_day.index.rename('ts', inplace= True)
#    #open
#    open_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Open
#    open_day.index.rename('ts', inplace= True)
#    #high
#    high_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.High
#    high_day.index.rename('ts', inplace= True)
#    #low
#    low_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Low
#    low_day.index.rename('ts', inplace= True)
#    #volume
#    volume_day['{}'.format(stocks_table[stocks_table.code == code].symbol.to_list()[0])] = df_day.Volume
#    volume_day.index.rename('ts', inplace= True)
  
  
  close_5min.reset_index().to_gbq(f'{dataset}.close_5min', project_id= project_id, if_exists='append')
#  open_5min.reset_index().to_gbq(f'{dataset}.open_5min', project_id= project_id, if_exists='append')
#  high_5min.reset_index().to_gbq(f'{dataset}.high_5min', project_id= project_id, if_exists='append')
#  low_5min.reset_index().to_gbq(f'{dataset}.low_5min', project_id= project_id, if_exists='append')
#  volume_5min.reset_index().to_gbq(f'{dataset}.volume_5min', project_id= project_id, if_exists='append')
  display(close_5min)
#
#  close_day.reset_index().to_gbq(f'{dataset}_day.close', project_id= project_id, if_exists='append')
#  open_day.reset_index().to_gbq(f'{dataset}_day.open', project_id= project_id, if_exists='append')
#  high_day.reset_index().to_gbq(f'{dataset}_day.high', project_id= project_id, if_exists='append')
#  low_day.reset_index().to_gbq(f'{dataset}_day.low', project_id= project_id, if_exists='append')
#  volume_day.reset_index().to_gbq(f'{dataset}_day.volume', project_id= project_id, if_exists='append')

  print('已完成{}項'.format(n))
print('Done')

##下載K棒資料並存入GBQ

In [ ]:
database_name = kbar_data
count = 0
for exchange in stock:
    for stocks in exchange:
        if stocks.exchange in (Exchange.TSE, Exchange.OTC):
            if stocks.code in set(existing_code): #跳過已經下載過的資料
                continue 
            if stocks.category == '00' or stocks.category == '':
                continue
            elif pd.to_datetime(stocks.update_date) != LAST_TRADE_DATE:
                continue
            elif re.search('[A-Z]', stocks.code) is None:  
              count += 1
              print(f'{count}. start download <{stocks.code} {stocks.name}> kbar data...')
              kbars = api.kbars(stocks, start='2018-12-07', end=TODAY)
              df = pd.DataFrame({**kbars})
              df.ts = pd.to_datetime(df.ts)
              df['Code'] = stocks.code
              df['Name'] = stocks.name
              df.to_gbq(destination_table= 'my_database.{}'.format(database_name), project_id= project_id, if_exists='append')
              print(f'<{stocks.code} {stocks.name}> kbar data is stored to Google BigQuery' + ' 目前進度{:.2f}%'.format((count/(len(df_stock) - len(existing_code)))*100))

## Resample

In [ ]:
from shioaji.constant import Exchange
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re

from talib.abstract import *
import numpy as np

contracts = api.Contracts
stock = contracts.Stocks
future = contracts.Futures

# 設定各頻率資料表名稱
table_name_5min = 'my_database.kbars_5min'
table_name_day = 'my_database.kbar_day'
table_name_week = 'my_database.kbar_week'
#table_name_last_15 = 'my_database.kbars_last_15'

# 傳入股票代碼，產生5分K資料並存至資料庫

def kbar_5min(stock_code, start = "2018-12-07"):
    print(f'generate 5minK for stock:<{stock_code}> {stock[stock_code].name}') #輸出程式執行點資料至console
    df = pd.DataFrame({**api.kbars(stock[stock_code], start= start,)}).set_index('ts')
    ##pd_gbq.read_gbq('SELECT Code, Name, Open, High, Low, Close, Volume, ts FROM my_database.{} WHERE Code = "{}"'.format(table_name, stock_code) , project_id= project_id, index_col='ts')
    df.index = pd.to_datetime(df.index)
    df_5min_kbar = df.resample(rule='5Min', closed='right').agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'}).dropna() #須去除NA，否則可能出現格式錯誤
    df_5min_kbar['Code'] = stock_code
    df_5min_kbar['Name'] = stock[stock_code].name
    # 技術指標
#    close = df_5min_kbar.Close
#    high = df_5min_kbar.High
#    low = df_5min_kbar.Low
#    df_5min_kbar['MA5'] = close.rolling(5).mean()
#    df_5min_kbar['MA10'] = close.rolling(10).mean()
#    df_5min_kbar['MA20'] = close.rolling(20).mean()
#    df_5min_kbar['MA60'] = close.rolling(60).mean()
#    df_5min_kbar['UB'] = BBANDS(close, 20, 2.0, 2.0)[0]
#    df_5min_kbar['MB'] = BBANDS(close, 20, 2.0, 2.0)[1]
#    df_5min_kbar['LB'] = BBANDS(close, 20, 2.0, 2.0)[2]


    df_5min_kbar.fillna(0, inplace= True)
    df_5min_kbar.sort_index()

    #last_15 = pd.DataFrame(close[(close.index.hour == 13)&(close.index.minute >= 15)]) # 晚於每個交易日13點15分的資料
    #last_15['Code'] = stock[stock_code].code
    #last_15['Name'] = stock[stock_code].name

    df_5min_kbar.reset_index(inplace= True)
    #last_15.reset_index(inplace= True)
    df_5min_kbar.to_gbq(f'{table_name_5min}', project_id= project_id, if_exists='append')
    #last_15.to_gbq(f'{table_name_last_15}', project_id= project_id, if_exists='append')
    print(f'stock:{stock_code}, 5minK is added to DB...') #輸出程式執行點資料至console

def kbar_day(stock_code, start = "2018-12-07"):
    print(f'generate DayK for stock:<{stock_code}> {stock[stock_code].name}') #輸出程式執行點資料至console
    df = pd.DataFrame({**api.kbars(stock[stock_code], start= start)}).set_index('ts')
    ##pd_gbq.read_gbq('SELECT Code, Name, Open, High, Low, Close, Volume, ts FROM my_database.{} WHERE Code = "{}"'.format(table_name, stock_code) , project_id= project_id, index_col='ts')
    df.index = pd.to_datetime(df.index)
    df_day_kbar = df.resample(rule='D').agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'}).dropna()
    df_day_kbar['Code'] = stock_code
    df_day_kbar['Name'] = stock[stock_code].name
    # 技術指標
#    close = df_day_kbar.Close
#    high = df_day_kbar.High
#    low = df_day_kbar.Low
#    df_day_kbar['MA5'] = close.rolling(5).mean()
#    df_day_kbar['MA10'] = close.rolling(10).mean()
#    df_day_kbar['MA20'] = close.rolling(20).mean()
#    df_day_kbar['MA60'] = close.rolling(60).mean()
#    df_day_kbar['UB'] = BBANDS(close, 20, 2.0, 2.0)[0]
#    df_day_kbar['MB'] = BBANDS(close, 20, 2.0, 2.0)[1]
#    df_day_kbar['LB'] = BBANDS(close, 20, 2.0, 2.0)[2]
    
    df_day_kbar.fillna(0, inplace= True)
    df_day_kbar.dropna(axis=0, inplace=True)
    df_day_kbar.sort_index()
    df_day_kbar.reset_index(inplace= True)
    df_day_kbar.to_gbq(f'{table_name_day}', project_id= project_id, if_exists='append',)
    print(f'stock:{stock_code}, DayK is added to DB...') #輸出程式執行點資料至console

def kbar_week(stock_code, start= "2018-12-07"):
    print(f'generate week_K for stock:<{stock_code}> {stock[stock_code].name}') #輸出程式執行點資料至console
    df = pd.DataFrame({**api.kbars(stock[stock_code], start= start)}).set_index('ts')
    ##pd_gbq.read_gbq('SELECT Code, Name, Open, High, Low, Close, Volume, ts FROM my_database.{} WHERE Code = "{}"'.format(table_name, stock_code) , project_id= project_id, index_col='ts')
    df.index = pd.to_datetime(df.index)
    df_week_kbar = df.resample(rule='W').agg({
        'Open': 'first', 
        'High': 'max', 
        'Low': 'min', 
        'Close': 'last',
        'Volume': 'sum'}).dropna()
    df_week_kbar['Code'] = stock_code
    df_week_kbar['Name'] = stock[stock_code].name

#    close = df_week_kbar.Close
#    high = df_week_kbar.High
#    low = df_week_kbar.Low
#    df_week_kbar['MA5'] = close.rolling(5).mean()
#    df_week_kbar['MA10'] = close.rolling(10).mean()
#    df_week_kbar['MA20'] = close.rolling(20).mean()
#    df_week_kbar['MA60'] = close.rolling(60).mean()
#    df_week_kbar['UB'] = BBANDS(close, 20, 2.0, 2.0)[0]
#    df_week_kbar['MB'] = BBANDS(close, 20, 2.0, 2.0)[1]
#    df_week_kbar['LB'] = BBANDS(close, 20, 2.0, 2.0)[2]
    
    df_week_kbar.fillna(0, inplace= True)
    df_week_kbar.dropna(axis=0, inplace=True)
    df_week_kbar.sort_index()
    df_week_kbar.reset_index(inplace= True)
    df_week_kbar.to_gbq(f'{table_name_week}', project_id= project_id, if_exists='append',)
    print(f'stock:{stock_code}, week_K is added to DB...') #輸出程式執行點資料至console

In [ ]:
from datetime import datetime, timedelta

start = '2018-12-07'
TODAY = datetime.today().strftime("%Y-%m-%d")

LAST_TRADE_DATE = max(df_stock.update_date)
list_code = []
update_date = []
for exchange in stock:
    for stocks in exchange:
        if stocks.exchange in (Exchange.TSE, Exchange.OTC):
          if stocks.category == '00' or stocks.category == '':
                continue
          elif stocks.update_date != LAST_TRADE_DATE: #剔除已下市股票
                continue
          elif re.search('[A-Z]', stocks.code) is None:
            list_code.append(stocks.code)
            update_date.append(stocks.update_date)
#list_code = list_stock.Code
LAST_TRADE_DATE = max(update_date)
df = pd.DataFrame([], index= None, columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Code'])

#-------------------5min----------------------
try:
  existing_kbar_5min = pd_gbq.read_gbq(
    f'''                #3個引號才有辦法分行讀
    SELECT Code, 
    Max(ts) as last_update, 
#    Min(ts) as init_update 
    FROM {table_name_5min} 
    GROUP BY Code
    '''
    , project_id= project_id,
    dtypes= [{'name': 'code', 'type': 'STRING'}, {'name': 'last_update', 'type': 'date'}]) 
except:
  existing_kbar_5min = df

#-------------------day----------------------
try:
  existing_kbar_day = pd_gbq.read_gbq(
    f'''
    SELECT Code, 
    Max(ts) as last_update, 
#    Min(ts) as init_update 
    FROM {table_name_day} 
    GROUP BY Code
    '''
    , project_id= project_id,
    dtypes= [{'name': 'code', 'type': 'STRING'}, {'name': 'last_update', 'type': 'date'}])
except:
  existing_kbar_day = df

#-------------------week----------------------
try:
  existing_kbar_week = pd_gbq.read_gbq(
    f'''
    SELECT Code, 
    Max(ts) as last_update, 
#    Min(ts) as init_update 
    FROM {table_name_week} 
    GROUP BY Code
    '''
    , project_id= project_id,
    dtypes= [{'name': 'code', 'type': 'STRING'}, {'name': 'last_update', 'type': 'date'}])
except:
  existing_kbar_week = df

Downloading: 100%|██████████| 1741/1741 [00:00<00:00, 5912.56rows/s]


In [ ]:
# 擷取所有欄位名稱(查一次要10MB，省著點用)
column_name= client.query(f'''
              SELECT
              * 
              FROM
              `{project_id}`.kbar.INFORMATION_SCHEMA.COLUMNS
              WHERE table_name="close_5min"''').to_dataframe().column_name.values.tolist()
print(column_name, '\n', '總共有{}項'.format(len(column_name[1:])))

['ts', 'OTC6612', 'OTC8034', 'OTC5487', 'OTC5386', 'OTC2745', 'OTC2724', 'OTC4735', 'OTC1569', 'OTC1565', 'OTC3594', 'OTC3083', 'OTC6570', 'OTC6210', 'OTC4726', 'OTC2752', 'OTC4533', 'OTC5223', 'OTC4113', 'OTC4534', 'OTC6169', 'OTC8087', 'OTC8099', 'OTC8076', 'OTC8437', 'OTC3228', 'OTC8424', 'OTC5543', 'OTC8908', 'OTC1336', 'OTC6527', 'OTC5475', 'OTC5864', 'OTC3562', 'OTC2729', 'OTC3088', 'OTC3252', 'OTC8418', 'OTC8071', 'OTC2755', 'OTC3265', 'OTC6568', 'OTC2718', 'OTC6624', 'OTC6245', 'OTC3260', 'OTC8084', 'OTC5480', 'OTC4974', 'OTC5016', 'OTC6425', 'OTC3587', 'OTC6712', 'OTC5488', 'OTC5281', 'OTC5014', 'OTC6247', 'OTC5383', 'OTC5324', 'OTC5601', 'OTC3611', 'OTC6163', 'OTC4138', 'OTC2067', 'OTC4609', 'OTC6419', 'OTC6227', 'OTC3564', 'OTC8085', 'OTC3360', 'OTC6109', 'OTC1777', 'OTC8435', 'OTC5468', 'OTC3556', 'OTC9950', 'OTC8930', 'OTC5353', 'OTC1570', 'OTC6589', 'OTC1333', 'OTC3379', 'OTC8077', 'OTC3630', 'OTC4430', 'OTC5287', 'OTC6752', 'OTC6576', 'OTC3548', 'OTC3323', 'OTC4147', 'OT

#捨不得刪

In [ ]:
#n= 0
#for code in stocks_table.code:
#  n += 1
#  kbars = api.kbars(api.Contracts.Stocks[f'{code}'], start=start)
#  df = pd.DataFrame({**kbars}, index= pd.to_datetime(kbars.ts)).drop('ts', axis= 1)
#  df_5min = df.resample(rule='5Min', closed='right').agg({
#        'Open': 'first', 
#        'High': 'max', 
#        'Low': 'min', 
#        'Close': 'last',
#        'Volume': 'sum'})
#  df_day = df.resample(rule='D').agg({
#        'Open': 'first', 
#        'High': 'max', 
#        'Low': 'min', 
#        'Close': 'last',
#        'Volume': 'sum'})
#  df_week = df.resample(rule='W').agg({
#        'Open': 'first', 
#        'High': 'max', 
#        'Low': 'min', 
#        'Close': 'last',
#        'Volume': 'sum'})
#  close1_5min = df_5min.Close.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  close1_day = df_day.Close.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  close1_week = df_week.Close.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#
#  open1_5min = df_5min.Open.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  open1_day = df_day.Open.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  open1_week = df_week.Open.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#
#  high1_5min = df_5min.High.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  high1_day = df_day.High.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  high1_week = df_week.High.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#
#  low1_5min = df_5min.Low.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  low1_day = df_day.Low.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  low1_week = df_week.Low.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#
#  volume1_5min = df_5min.Volume.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  volume1_day = df_day.Volume.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  volume1_week = df_week.Volume.rename(stocks_table[stocks_table.code == code].name.to_list()[0])
#  try:
#    close2_5min = pd.concat([close2_5min, close1_5min], axis=1)
#    close2_day = pd.concat([close2_day, close1_day], axis=1)
#    close2_week = pd.concat([close2_week, close1_week], axis=1)
#
#    open2_5min = pd.concat([open2_5min, open1_5min], axis=1)
#    open2_day = pd.concat([open2_day, open1_day], axis=1)
#    open2_week = pd.concat([open2_week, open1_week], axis=1)
#
#    high2_5min = pd.concat([high2_5min, high1_5min], axis=1)
#    high2_day = pd.concat([high2_day, high1_day], axis=1)
#    high2_week = pd.concat([high2_week, high1_week], axis=1)
#
#    low2_5min = pd.concat([low2_5min, low1_5min], axis=1)
#    low2_day = pd.concat([low2_day, low1_day], axis=1)
#    low2_week = pd.concat([low2_week, low1_week], axis=1)
#
#    volume2_5min = pd.concat([volume2_5min, volume1_5min], axis=1)
#    volume2_day = pd.concat([volume2_day, volume1_day], axis=1)
#    volume2_week = pd.concat([volume2_week, volume1_week], axis=1)
#  except:
#    close2_5min = close1_5min
#    close2_day = close1_day
#    close2_week = close1_week
#
#    open2_5min = open1_5min
#    open2_day = open1_day
#    open2_week = open1_week
#
#    high2_5min = high1_5min
#    high2_day = high1_day
#    high2_week = high1_week
#
#    low2_5min = low1_5min
#    low2_day = low1_day
#    low2_week = low1_week
#
#    volume2_5min = volume1_5min
#    volume2_day = volume1_day
#    volume2_week = volume1_week
#  print('還有{}項'.format(len(stocks_table.code) - n))
#
#
##close
#close2_5min.fillna(0, inplace= True)
#close2_5min.sort_index()
#close2_5min.reset_index(inplace= True)
#close2_5min.to_gbq('close_5min', project_id= project_id, if_exists='append')
#
#close2_day.fillna(0, inplace= True)
#close2_day.sort_index()
#close2_day.reset_index(inplace= True)
#close2_day.to_gbq('close_day', project_id= project_id, if_exists='append')
#
#close2_week.fillna(0, inplace= True)
#close2_week.sort_index()
#close2_week.reset_index(inplace= True)
#close2_week.to_gbq('close_week', project_id= project_id, if_exists='append')
#
##open
#open2_5min.fillna(0, inplace= True)
#open2_5min.sort_index()
#open2_5min.reset_index(inplace= True)
#open2_5min.to_gbq('open_5min', project_id= project_id, if_exists='append')
#
#open2_day.fillna(0, inplace= True)
#open2_day.sort_index()
#open2_day.reset_index(inplace= True)
#open2_day.to_gbq('open_day', project_id= project_id, if_exists='append')
#
#open2_week.fillna(0, inplace= True)
#open2_week.sort_index()
#open2_week.reset_index(inplace= True)
#open2_week.to_gbq('open_week', project_id= project_id, if_exists='append')
#
##high
#high2_5min.fillna(0, inplace= True)
#high2_5min.sort_index()
#high2_5min.reset_index(inplace= True)
#high2_5min.to_gbq('high_5min', project_id= project_id, if_exists='append')
#
#high2_day.fillna(0, inplace= True)
#high2_day.sort_index()
#high2_day.reset_index(inplace= True)
#high2_day.to_gbq('high_day', project_id= project_id, if_exists='append')
#
#high2_week.fillna(0, inplace= True)
#high2_week.sort_index()
#high2_week.reset_index(inplace= True)
#high2_week.to_gbq('high_week', project_id= project_id, if_exists='append')
#
##low
#low2_5min.fillna(0, inplace= True)
#low2_5min.sort_index()
#low2_5min.reset_index(inplace= True)
#low2_5min.to_gbq('low_5min', project_id= project_id, if_exists='append')
#
#low2_day.fillna(0, inplace= True)
#low2_day.sort_index()
#low2_day.reset_index(inplace= True)
#low2_day.to_gbq('low_day', project_id= project_id, if_exists='append')
#
#low2_week.fillna(0, inplace= True)
#low2_week.sort_index()
#low2_week.reset_index(inplace= True)
#low2_week.to_gbq('low_week', project_id= project_id, if_exists='append')
#
##volume
#volume2_5min.fillna(0, inplace= True)
#volume2_5min.sort_index()
#volume2_5min.reset_index(inplace= True)
#volume2_5min.to_gbq('volume_5min', project_id= project_id, if_exists='append')
#
#volume2_day.fillna(0, inplace= True)
#volume2_day.sort_index()
#volume2_day.reset_index(inplace= True)
#volume2_day.to_gbq('volume_day', project_id= project_id, if_exists='append')
#
#volume2_week.fillna(0, inplace= True)
#volume2_week.sort_index()
#volume2_week.reset_index(inplace= True)
#volume2_week.to_gbq('volume_week', project_id= project_id, if_exists='append')

In [ ]:
#for code in list_code:
#
#  code_5min = code
#  code_day = code
#  code_week = code
#
#  if code_5min in set(existing_kbar_5min.Code):
#    number_5min = ((pd.Series(list_code)[pd.Series(list_code) == code_5min]).index[0]) + 1
#    if pd.to_datetime(existing_kbar_5min[existing_kbar_5min.Code == code_5min].last_update.to_list()) < LAST_TRADE_DATE:
#      kbar_5min(code_5min, start = ([*existing_kbar_5min[existing_kbar_5min.Code == code_5min].last_update][0] + timedelta(days= 1)).strftime('%Y-%m-%d'))
#      print('[已有資料]還剩下{:,}項'.format((len(list_code) - number_5min)))
#    else:
#      pass
#  elif code_5min not in set(existing_kbar_5min.Code):
#    kbar_5min(code_5min)
#    number_5min = ((pd.Series(list_code)[pd.Series(list_code) == code]).index[0]) + 1
#    print('還剩下{:,}項'.format((len(list_code) - number_5min)))
#
#  if code_day in set(existing_kbar_day.Code):
#    number_day = ((pd.Series(list_code)[pd.Series(list_code) == code_day]).index[0]) + 1
#    if pd.to_datetime(existing_kbar_day[existing_kbar_day.Code == code_day].last_update.to_list()) < LAST_TRADE_DATE:
#      kbar_day(code_day, start = ([*existing_kbar_day[existing_kbar_day.Code == code_day].last_update][0] + timedelta(days= 1)).strftime('%Y-%m-%d'))
#      print('[已有資料]還剩下{:,}項'.format(len(list_code) - len(existing_kbar_day.Code) - number_day))
#    else:
#      pass
#  elif code_day not in set(existing_kbar_day.Code):
#    number_day = ((pd.Series(list_code)[pd.Series(list_code) == code_day]).index[0]) + 1
#    kbar_day(code_day)
#    print('還剩下{:,}項'.format(len(list_code) - len(existing_kbar_day) - number_day))
#  
#  if code_week in set(existing_kbar_week.Code):
#    number_week = ((pd.Series(list_code)[pd.Series(list_code) == code_week]).index[0]) + 1
#    if pd.to_datetime(existing_kbar_week[existing_kbar_week.Code == code_week].last_update.to_list()) < LAST_TRADE_DATE:
#      kbar_week(code_week, start = ([*existing_kbar_week[existing_kbar_week.Code == code_week].last_update][0] + timedelta(days= 1)).strftime('%Y-%m-%d'))
#      print('[已有資料]還剩下{:,}項'.format(len(list_code) - len(existing_kbar_week.Code)- number_week))
#    else:
#      pass
#  elif code_week not in set(existing_kbar_week.Code):
#    number_week = ((pd.Series(list_code)[pd.Series(list_code) == code_week]).index[0]) + 1
#    kbar_week(code_week)
#    print('還剩下{:,}項'.format(len(list_code) - len(existing_kbar_week) - number_week))
#
#print('Done')